**Important note:** `Cxx.jl` unfortunately only works with julia 1.3.1 :(

# Isn't multiple dispatch just function overloading?

Answer: No. Static types vs actual dynamic types.

In [ ]:
abstract type Pet end

struct Dog <: Pet
    name::String
end
struct Cat <: Pet
    name::String
end

function encounter(a::Pet, b::Pet)
    verb = meets(a,b)
    println("$(a.name) meets $(b.name) and $verb.")
end

meets(a::Dog, b::Dog) = "sniffs"
meets(a::Dog, b::Cat) = "chases"
meets(a::Cat, b::Dog) = "hisses"
meets(a::Cat, b::Cat) = "slinks"

In [ ]:
fido     = Dog("fido")
rex      = Dog("rex")
whiskers = Cat("whiskers")
spots    = Cat("spots")

encounter(fido, rex);
encounter(fido, whiskers);
encounter(whiskers, rex);
encounter(whiskers, spots);

### Alright, let's try the same in, say, C++

```C++
#include <iostream>
#include <string>

using namespace std;

class Pet {
    public:
        string name;
};

string meets(Pet a, Pet b) { return "FALLBACK"; }

void encounter(Pet a, Pet b) {
    string verb = meets(a,b);
    cout << a.name << " meets " << b.name << " and " << verb << endl;
}

class Dog : public Pet {};
class Cat : public Pet {};

string meets(Dog a, Dog b) { return "sniffs"; }
string meets(Dog a, Cat b) { return "chases"; }
string meets(Cat a, Dog b) { return "hisses"; }
string meets(Cat a, Cat b) { return "slinks"; }

int main() {
    Dog fido;     fido.name = "Fido";
    Dog rex;      rex.name = "Rex";
    Cat whiskers; whiskers.name = "Whiskers";
    Cat spots;    spots.name = "Spots";

    encounter(fido, rex);
    encounter(fido, whiskers);
    encounter(whiskers, rex);
    encounter(whiskers, spots);

    return 0;
}
```

### What do you think this does?##

<details>
  <summary>Click to reveal (if Cxx doesn't work for you)</summary>
<br>
    
```
carsten@l91:~/Desktop/Oulu2020/backup/function_overloading$ clang++ pets.cpp -o pets && ./pets

Fido meets Rex and FALLBACK
Fido meets Whiskers and FALLBACK
Whiskers meets Rex and FALLBACK
Whiskers meets Spots and FALLBACK
```
</details>

In [ ]:
using Cxx

In [ ]:
cxx"""
#include <iostream>
#include <string>

using namespace std;

class Pet {
    public:
        string name;
};

string meets(Pet a, Pet b) { return "FALLBACK"; }

void encounter(Pet a, Pet b) {
    string verb = meets(a,b);
    cout << a.name << " meets " << b.name << " and " << verb << endl;
}

class Dog : public Pet {};
class Cat : public Pet {};

string meets(Dog a, Dog b) { return "sniffs"; }
string meets(Dog a, Cat b) { return "chases"; }
string meets(Cat a, Dog b) { return "hisses"; }
string meets(Cat a, Cat b) { return "slinks"; }

int main() {
    Dog fido;     fido.name = "Fido";
    Dog rex;      rex.name = "Rex";
    Cat whiskers; whiskers.name = "Whiskers";
    Cat spots;    spots.name = "Spots";

    encounter(fido, rex);
    encounter(fido, whiskers);
    encounter(whiskers, rex);
    encounter(whiskers, spots);

    return 0;
}
"""

In [ ]:
@cxx main()

**What?! Why?**

Templates get you a bit further, but the type information is still only static.

## Dynamic dispatch

* In Julia, type computation can happen dynamically (at runtime) or statically (at JIT compile time).

* The static type computation is equivalent to the C++ version

In [ ]:
A1 = [1, 2.0, [3,4]]

In [ ]:
using LinearAlgebra
norm.(A1) # norm function is chosen at runtime, depending on actual content of the array

In C++, this could be realized by using a common base class and a virtual norm function. Demonstrating this in a simple example:

In [ ]:
using Cxx

In [ ]:
cxx"""
#include <iostream>
using namespace std;

struct A {
   void f() { cout << "Class A" << endl; }
};

struct B: A {
   void f() { cout << "Class B" << endl; }
};

void g(A& arg) {
   arg.f();
}

int main() {
   B x;
   g(x);
}
"""

In [ ]:
@cxx main(); # static dispatch

In [ ]:
cxx"""
#include <iostream>
using namespace std;

struct A {
   virtual void f() { cout << "Class A" << endl; }
};

struct B: A {
   void f() { cout << "Class B" << endl; }
};

void g(A& arg) {
   arg.f();
}

int main() {
   B x;
   g(x);
}
"""

In [ ]:
@cxx main(); # dynamic dispatch

* A virtual function cannot be global.
* `arg.f()` is single dispatch

In Julia:

In [ ]:
abstract type A end
struct B <: A end

f(::A) = "Class A"
f(::B) = "Class B"

g(arg::A) = f(arg)

x = B()
g(x)

Julia implements *multiple dispatch*:

In [ ]:
A1 = [1, 2.0, "hello"]

In [ ]:
A2 = [4f0, 5im, "world"]

In [ ]:
A1 .* A2 # dynamic dispatch based on both(!) arguments to *